In [2]:
import plotly.express as px
import pandas as pd
from time import time
from tqdm import tqdm
import numpy as np

In [3]:
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [9]:
from model.klotski import Klotski
game = Klotski()

a_star_results = []
with tqdm(total=4*4*4*3) as pbar:
    for mm in [1, 6, 12, 20]:
        for zm in [1, 6, 12, 20]: 
            for im in [1, 6, 12, 20]:
                for lm in [1.0, 0.5, 0.2]:
                    
                    game.read_board('inputs/set1/board11.txt')
                    try:
                        with time_limit(8):
                            start = time()
                            result = game.a_star(mm, zm, im, lm)
                            end = time()
                            exec_time = end - start
                            depth = len(result.move_history)
                    except TimeoutException as e:
                        exec_time = np.inf
                        depth = np.inf

                    a_star_results.append({
                        'mm': mm, 'zm': zm, 'im': im, 'lm': lm,
                        'exec_time': exec_time, 'result_depth': depth
                    })

                    pbar.update()
                
df_a_star = pd.DataFrame(a_star_results)

100%|██████████| 625/625 [1:23:10<00:00,  7.98s/it]


In [10]:
df = df_a_star
df[df['exec_time'] != np.inf].sort_values('exec_time', ascending=True)

,mm,zm,im,lm,exec_time,result_depth
220,4,16,32,1.00,3.704200,54.0
221,4,16,32,0.67,4.160043,54.0
67,1,8,16,0.33,5.597938,54.0
191,4,8,16,0.67,6.275105,54.0
445,16,8,32,1.00,6.341387,54.0
37,1,4,8,0.33,6.755980,54.0
192,4,8,16,0.33,6.857281,54.0
256,8,1,4,0.67,7.019669,50.0
280,8,4,4,1.00,7.468893,50.0
251,8,1,1,0.67,7.866697,50.0
